In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from statsmodels.stats.outliers_influence import variance_inflation_factor
import random
from tqdm.notebook import tqdm

random.seed(42)
pd.set_option('display.max_columns', None)

In [48]:
def data_processing(train_data, test_data):
    # Drop the specified columns
    columns_to_drop = ['Alley', 'PoolQC', 'MiscFeature', 'Fence']
    train_data.drop(columns_to_drop, axis=1, inplace=True)
    test_data.drop(columns_to_drop, axis=1, inplace=True)

    # Select relevant features
    features = ['LotFrontage', 'LotArea', 'Neighborhood']
    data_train = train_data[features]
    data_train = pd.get_dummies(data_train)

    # Standardize the data
    scaler = StandardScaler()
    data_scaled_train = scaler.fit_transform(data_train)

    # KNN Imputer for train data
    imputer = KNNImputer(n_neighbors=5)
    data_imputed_train = imputer.fit_transform(data_scaled_train)

    # Convert the scaled data back to the original scale for train data
    data_imputed_original_scale_train = scaler.inverse_transform(data_imputed_train)

    # Update LotFrontage in train_data
    train_data['LotFrontage'] = data_imputed_original_scale_train[:, data_train.columns.get_loc('LotFrontage')]

    # Process test data 
    data_test = test_data[features]
    data_test = pd.get_dummies(data_test, columns=['Neighborhood'])

    # Align columns of test data with train data
    data_test = data_test.reindex(columns = data_train.columns, fill_value=0)

    # Standardize test data
    data_scaled_test = scaler.transform(data_test)

    # KNN Imputer for test data
    imputer_test = KNNImputer(n_neighbors=5)
    data_imputed_test = imputer_test.fit_transform(data_scaled_test)

    # Convert the scaled data back to the original scale for test data
    data_imputed_test_scale = scaler.inverse_transform(data_imputed_test)

    # Update LotFrontage in test_data
    test_data['LotFrontage'] = data_imputed_test_scale[:, data_test.columns.get_loc('LotFrontage')]

    # Fill missing values for 'GarageYrBlt' and 'MasVnrArea' with zeros
    train_data['GarageYrBlt'].fillna(0, inplace=True)
    test_data['GarageYrBlt'].fillna(0, inplace=True)
    train_data['MasVnrArea'].fillna(0, inplace=True)
    test_data['MasVnrArea'].fillna(0, inplace=True)

    # Identify numerical features and fill missing values with their median
    numerical_features = train_data.select_dtypes(include=[np.number])
    numerical_features.fillna(numerical_features.median(), inplace=True)
    numerical_features_test = test_data.select_dtypes(include=[np.number])
    numerical_features_test.fillna(numerical_features_test.median(), inplace=True)
    
    # Dropping features with low or negative correlations
    features_to_drop = [
        'KitchenAbvGr', 'EnclosedPorch', 
        'OverallCond', 'LowQualFinSF', 
        'MSSubClass', 'Unnamed: 0', 
        'Id', 'MiscVal',
        'YrSold','LotFrontage',
        'BsmtHalfBath', 'BsmtFinSF2'
    ]
    # Dropping the features from the DataFrame
    test_data.drop(features_to_drop, axis=1, inplace=True)
    train_data.drop(features_to_drop, axis=1, inplace=True)
    
    features_to_drop = [
         'GarageArea' ,'TotalBsmtSF','TotRmsAbvGrd'
    ]
    train_data.drop(features_to_drop, axis=1, inplace=True)
    test_data.drop(features_to_drop, axis=1, inplace=True)
    
    categorical_cols_with_missing = ['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
    # Fill missing values with the most common value in each categorical column
    for col in categorical_cols_with_missing:
        mode_value = train_data[col].mode()[0]  # Get the mode value for the column
        train_data[col].fillna(mode_value, inplace=True)
    categorical_cols_with_missing_test = ['MasVnrType', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'FireplaceQu', 'GarageType', 'GarageQual']
    for col in categorical_cols_with_missing_test:
        mode_value = test_data[col].mode()[0]
        test_data[col].fillna(mode_value, inplace=True)
        
    features_to_drop = [
         'Street' ,'Utilities','LotConfig','Condition2','BldgType','HouseStyle','RoofMatl',
        'Exterior1st','Exterior2nd','ExterCond','BsmtCond','Heating','HeatingQC','CentralAir',
        'Electrical','Functional','GarageCond','GarageFinish','PavedDrive','BsmtFinType2'
    ]
    train_data.drop(features_to_drop, axis=1, inplace=True)
    test_data.drop(features_to_drop, axis=1, inplace=True)
    
    lot_shape_mapping = {
        'Reg': 1,
        'IR1': 2,
        'IR2': 3,
        'IR3': 4
    }

    # Apply the mapping to your DataFrame
    train_data['LotShape_Ordinal'] = train_data['LotShape'].map(lot_shape_mapping)
    # Create the interaction term
    train_data['LotArea_Shape_Interaction'] = train_data['LotArea'] * train_data['LotShape_Ordinal']
    # Drop the original 'LotArea' and 'LotShape' columns
    train_data.drop(['LotArea', 'LotShape'], axis=1, inplace=True)
    test_data['LotShape_Ordinal'] = test_data['LotShape'].map(lot_shape_mapping)
    test_data['LotArea_Shape_Interaction'] = test_data['LotArea'] * test_data['LotShape_Ordinal']
    test_data.drop(['LotArea', 'LotShape'], axis=1, inplace=True)
    
    #Drop '1stFlrSF ’，‘2ndFlrSF’
    train_data.drop(['1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
    test_data.drop(['1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
    
    train_data['TotalBsmtSF'] = train_data['BsmtFinSF1'] + train_data['BsmtUnfSF']
    train_data.drop(['BsmtFinSF1', 'BsmtUnfSF'], axis=1, inplace=True)
    test_data['TotalBsmtSF'] = test_data['BsmtFinSF1'] + test_data['BsmtUnfSF']
    test_data.drop(['BsmtFinSF1', 'BsmtUnfSF'], axis=1, inplace=True)
    
    # Calculate the total number of bathrooms
    test_data['TotalBaths'] = test_data['BsmtFullBath'] + test_data['FullBath'] + (test_data['HalfBath'] * 0.5)
    test_data.drop(['BsmtFullBath', 'FullBath', 'HalfBath'], axis=1, inplace=True)
    train_data['TotalBaths'] = train_data['BsmtFullBath'] + train_data['FullBath'] + (train_data['HalfBath'] * 0.5)
    train_data.drop(['BsmtFullBath', 'FullBath', 'HalfBath'], axis=1, inplace=True)
    
    train_data.drop(['3SsnPorch', 'MoSold', 'ScreenPorch'], axis=1, inplace=True)
    test_data.drop(['3SsnPorch', 'MoSold', 'ScreenPorch'], axis=1, inplace=True)
    
    # Binary encode the 'RoofStyle' column where '1' represents 'Gable' or 'Hip' roofs and '0' all others
    train_data['RoofStyle_Binary'] = train_data['RoofStyle'].apply(lambda x: 1 if x in ['Gable', 'Hip'] else 0)
    # Optionally, drop the original 'RoofStyle' column if you no longer need it
    train_data.drop('RoofStyle', axis=1, inplace=True)
    
    test_data['RoofStyle_Binary'] = test_data['RoofStyle'].apply(lambda x: 1 if x in ['Gable', 'Hip'] else 0)
    test_data.drop('RoofStyle', axis=1, inplace=True)
    
    train_data['MasVnrType_BrkFace'] = train_data['MasVnrType'].apply(lambda x: 1 if x == 'BrkFace' else 0)
    train_data.drop('MasVnrType', axis=1, inplace=True)
    
    test_data['MasVnrType_BrkFace'] = test_data['MasVnrType'].apply(lambda x: 1 if x == 'BrkFace' else 0)
    test_data.drop('MasVnrType', axis=1, inplace=True)
    
    # Define a mapping from quality ratings to numbers
    quality_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}

    # Apply ordinal encoding
    for feature in ['ExterQual', 'BsmtQual', 'FireplaceQu', 'GarageQual','KitchenQual']:
        train_data[feature] = train_data[feature].map(quality_mapping)
    for feature in ['ExterQual', 'BsmtQual', 'FireplaceQu', 'GarageQual','KitchenQual']:
        test_data[feature] = test_data[feature].map(quality_mapping)
        
    land_contour_mapping = {'Lvl': 4, 'Bnk': 3, 'HLS': 2, 'Low': 1}
    train_data['LandContour'] = train_data['LandContour'].map(land_contour_mapping)
    test_data['LandContour'] = test_data['LandContour'].map(land_contour_mapping)
    
    land_slope_mapping = {'Gtl': 3, 'Mod': 2, 'Sev': 1}
    train_data['LandSlope'] = train_data['LandSlope'].map(land_slope_mapping)
    test_data['LandSlope'] = test_data['LandSlope'].map(land_slope_mapping)
    
    ordinal_mapping = {
        'No': 0,   # No Basement
        'Unf': 1,  # Unfinished
        'LwQ': 2,  # Low Quality
        'Rec': 3,  # Average Quality
        'BLQ': 4,  # Below Average Quality
        'ALQ': 5,  # Average Living Quarters
        'GLQ': 6   # Good Living Quarters
    }

    # Apply the mapping to the 'BsmtFinType1' column
    train_data['BsmtFinType1'] = train_data['BsmtFinType1'].map(ordinal_mapping)
    test_data['BsmtFinType1'] = test_data['BsmtFinType1'].map(ordinal_mapping)
    
    neighborhood_mapping = {
        'CollgCr':1, 'Edwards':2, 'NAmes':3, 'Somerst':4, 'Timber':5, 'Veenker':6,
           'OldTown':7, 'BrkSide':8, 'ClearCr':9, 'MeadowV':10, 'Mitchel':11, 'SawyerW':12,
           'StoneBr':13, 'NoRidge':14, 'SWISU':15, 'Sawyer':16, 'Gilbert':17, 'IDOTRR':18,
           'NridgHt':19, 'Blmngtn':20, 'NWAmes':21, 'Crawfor':22, 'BrDale':23, 'NPkVill':24
    }
    train_data['Neighborhood'] = train_data['Neighborhood'].map(neighborhood_mapping)
    neighborhood_mapping_test = {
        'NAmes':3, 'Somerst':4, 'Edwards':2, 'CollgCr':1, 'OldTown':7, 'StoneBr':13,
           'IDOTRR':18, 'Crawfor':22, 'Gilbert':17, 'NridgHt':19, 'BrkSide':8, 'Sawyer':16,
           'SWISU':15, 'NWAmes':21, 'NoRidge':14, 'Blmngtn':20, 'Mitchel':11, 'Blueste':25,
           'SawyerW':12, 'Timber':5, 'NPkVill':24, 'MeadowV':10, 'ClearCr':9, 'BrDale':23,
           'Veenker':6
    }
    test_data['Neighborhood'] = test_data['Neighborhood'].map(neighborhood_mapping_test)
    
    Condition1_mapping = {
        'Norm':1, 'Feedr':2, 'RRAn':3, 'Artery':4, 'RRAe':5, 'RRNn':6, 'PosN':7, 'RRNe':8,
           'PosA':9
    }
    test_data['Condition1'] = test_data['Condition1'].map(Condition1_mapping)
    train_data['Condition1'] = train_data['Condition1'].map(Condition1_mapping)
    
    Foundation_mapping = {
        'PConc':1, 'CBlock':2, 'BrkTil':3, 'Slab':4, 'Stone':5, 'Wood':6
    }
    test_data['Foundation'] = test_data['Foundation'].map(Foundation_mapping)
    train_data['Foundation'] = train_data['Foundation'].map(Foundation_mapping)
    
    BsmtExposure_mapping = {
        'No':0, 'Mn':1, 'Gd':2, 'Av':3
    }
    test_data['BsmtExposure'] = test_data['BsmtExposure'].map(BsmtExposure_mapping)
    train_data['BsmtExposure'] = train_data['BsmtExposure'].map(BsmtExposure_mapping)
    
    GarageType_mapping = {
        'Attchd':1, 'Detchd':2, 'BuiltIn':3, 'Basment':4, 'CarPort':5, '2Types':6
    }
    test_data['GarageType'] = test_data['GarageType'].map(GarageType_mapping)
    train_data['GarageType'] = train_data['GarageType'].map(GarageType_mapping)
    
    SaleType_mapping_train = {
        'WD':1, 'New':5, 'COD':3, 'CWD':8, 'ConLw':7, 'ConLD':2, 'ConLI':4, 'Oth':6, 'Con':9
    }
    train_data['SaleType'] = train_data['SaleType'].map(SaleType_mapping_train)
    SaleType_mapping_test = {
        'WD':1, 'ConLD':2, 'COD':3, 'ConLI':4, 'New':5, 'Oth':6, 'ConLw':7
    }
    test_data['SaleType'] = test_data['SaleType'].map(SaleType_mapping_test)
    
    SaleCondition_mapping = {
        'Normal':1, 'Partial':2, 'Abnorml':3, 'Family':4, 'Alloca':5, 'AdjLand':6
    }
    test_data['SaleCondition'] = test_data['SaleCondition'].map(SaleCondition_mapping)
    train_data['SaleCondition'] = train_data['SaleCondition'].map(SaleCondition_mapping)
    
    # Initialize the label encoder
    label_encoder = LabelEncoder()
    label_encoder_test = LabelEncoder()
    # Fit and transform the 'MSZoning' feature
    train_data['MSZoning'] = label_encoder.fit_transform(train_data['MSZoning'])
    test_data['MSZoning'] = label_encoder_test.fit_transform(test_data['MSZoning'])
    
    # Identify the columns to exclude from standardization
    cols_to_exclude = ['SalePrice']
    # Select only the numeric features by excluding the specified columns
    numeric_cols_test = test_data.select_dtypes(include=['number']).columns.tolist()
    cols_to_standardize = [col for col in numeric_cols_test if col not in cols_to_exclude]
    # Standardize the numeric columns that are not excluded
    scaler_testdata = StandardScaler()
    test_data[cols_to_standardize] = scaler.fit_transform(test_data[cols_to_standardize])

    numeric_cols_train = train_data.select_dtypes(include=['number']).columns.tolist()
    cols_to_standardize = [col for col in numeric_cols_test if col not in cols_to_exclude]
    scaler_traindata = StandardScaler()
    train_data[cols_to_standardize] = scaler.fit_transform(train_data[cols_to_standardize])

    return train_data, test_data

In [49]:
def test_data_prediction(train_data, test_data):
    
    train_data, test_data = data_processing(train_data, test_data)

    kmeans = KMeans(n_clusters=3, random_state=0)
    train_clusters = kmeans.fit_predict(train_data)
    train_data['Group'] = train_clusters

    ##### Train classifier #####

    X_train = train_data.drop('Group', axis=1)
    y_train = train_data['Group']
    clf = RandomForestClassifier(random_state=0)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [5, 10, 15],
        'min_samples_split': [2, 5, 10]
    }
    grid_search = GridSearchCV(clf, param_grid, cv=5, verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    test_data['Group'] = grid_search.predict(test_data)
    
    ##### predict on test data #####
    
    group_models = {}
    all_predictions = np.zeros(len(test_data))
    for group in tqdm(test_data['Group'].unique()):
        group_data_train = train_data[train_data['Group'] == group]
        X_train_group = group_data_train.drop(['SalePrice', 'Group'], axis=1)
        y_train_group = group_data_train['SalePrice']
        group_data_test = test_data[test_data['Group'] == group]
        X_test_group = group_data_test.drop(['SalePrice', 'Group'], axis=1)
        y_test_group = group_data_test['SalePrice']
        
        estimators = [
            ('lr', LinearRegression()),
            ('dt', DecisionTreeRegressor(random_state=42)),
            ('rf', RandomForestRegressor(random_state=42)),
            ('svr', SVR())
        ]
        
        stack_reg = StackingRegressor(estimators=estimators, final_estimator=RandomForestRegressor(random_state=42))
        stack_reg.fit(X_train_group, y_train_group)
        y_pred_group = stack_reg.predict(X_test_group)
        all_predictions[group_data_test.index] = y_pred_group

        # comparison_df = pd.DataFrame({
        #     'Actual': y_test_group,
        #     'Predicted': y_pred_group
        # })
        # comparison_df = comparison_df.reset_index(drop=True)
        # print('For group', group)
        # print(comparison_df)

        mse = mean_squared_error(y_test_group, y_pred_group)
        group_models[group] = {'model': stack_reg, 'mse': mse}

    for group, info in sorted(group_models.items()):
        print(f"Group {group}: MSE = {info['mse']}")
    overall_mse = mean_squared_error(test_data['SalePrice'], all_predictions)
    print("Overall MSE for the test dataset:", overall_mse)

    return test_data['SalePrice']

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
predicted_price = test_data_prediction(train_data, test_data)
predicted_price

Fitting 5 folds for each of 27 candidates, totalling 135 fits


  0%|          | 0/3 [00:00<?, ?it/s]

Group 0: MSE = 262980124.03118658
Group 1: MSE = 7619871022.58241
Group 2: MSE = 710767037.7397035
Overall MSE for the test dataset: 979819208.3504664


0      120500
1      155000
2      118000
3      188000
4      160000
        ...  
433    235000
434    170000
435    155000
436    395192
437    110500
Name: SalePrice, Length: 438, dtype: int64